In [ ]:
%pip install scikit-learn pandas seaborn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 31.6 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 46.2 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 42.0 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 32.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 51.6 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 57.0 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14/14 [seaborn]3/14 [seaborn]ib]n]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from modules.column import encode_columns, get_columns_by_types
from modules.fillers import fill_na_knn


In [84]:
df_init = sns.load_dataset('titanic')
df = df_init.copy()

In [ ]:
for column in encoders.keys():
    # float_column : pd.DataFrame = encoded_df[column+'_encoded'].astype('float64')
    # print(float_column)
    # print(encoded_df[column+'_encoded'])
    result = fill_na_knn(encoded_df.drop(columns=column), f"{column}_encoded", n_neighbors=5)

    # print(result)

KeyError: 'deck_encoded'

In [87]:
df[df.columns[df.isnull().any()]]

,age,embarked,deck,embark_town
0,22.0,S,NaN,Southampton
1,38.0,C,C,Cherbourg
2,26.0,S,NaN,Southampton
3,35.0,S,C,Southampton
4,35.0,S,NaN,Southampton
...,...,...,...,...
886,27.0,S,NaN,Southampton
887,19.0,S,B,Southampton
888,NaN,S,NaN,Southampton
889,26.0,C,C,Cherbourg


In [ ]:
df.notnull().all()

survived         True
pclass           True
sex              True
age             False
sibsp            True
parch            True
fare             True
embarked        False
class            True
who              True
adult_male       True
deck             True
embark_town     False
alive            True
alone            True
deck_decoded     True
dtype: bool

In [ ]:
def get_decoded_columns(encoded_df : pd.DataFrame, encoders: dict, embdedded=False) -> pd.DataFrame :
    if not embdedded:
        decoded_df = encoded_df.copy()
    else:
        decoded_df = pd.DataFrame()


    for column in encoded_df.columns:
        series = encoded_df[column]

        original_column_name = column.removesuffix('_encoded')
        encoder = encoders[original_column_name]
        
        decoded = series.map(lambda x: encoder.inverse_transform([int(x)])[0] if pd.notnull(x) else np.nan)
        decoded_df[original_column_name + '_decoded'] = decoded
       
    return decoded_df

# TODO: embed get_decoded_columns into the code
def pipeline_missing_values_category_values(df : pd.DataFrame) -> pd.DataFrame :

    filled_df = df.copy()
    # getting category columns with missing values
    null_columns : pd.DataFrame = filled_df[filled_df.columns[filled_df.isnull().any()]]
    null_columns : pd.DataFrame = get_columns_by_types(null_columns, ['category'])
    null_columns_names : list[str] = list(null_columns.columns)
    # encode all categorical columns with missing values
    encoded_df, encoders = encode_columns(filled_df, columns=null_columns_names, embedded=True).values()

    # encoders.keys() -> has only encoded column names
    for column in encoders.keys():
        
        # transforming to numerical value for KNNImputer
        encoded_df[column+'_encoded'] = encoded_df[column+'_encoded'].astype('Int64')

        # filling in the missing values 
        filled_encoded_df = fill_na_knn(encoded_df, column+'_encoded')

        # rounding up the values for the inverse_transform
        filled_encoded_df[column+'_encoded'] = filled_encoded_df[column+'_encoded'].astype('int64')
    
        # getting LabelEncoder() of the column
        column_encoder = encoders.get(column)
        
        # decoding to np.array
        column_decoded_array = column_encoder.inverse_transform(filled_encoded_df[column+'_encoded'])
        # updating the original df
        filled_df[column] = pd.DataFrame(column_decoded_array, columns=[column], dtype='category')


    return filled_df


In [ ]:
test = pipeline_missing_values_category_values(df)

In [90]:
test.info()

<class 'pandas.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    str     
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    str     
 8   class        891 non-null    category
 9   who          891 non-null    str     
 10  adult_male   891 non-null    bool    
 11  deck         891 non-null    category
 12  embark_town  889 non-null    str     
 13  alive        891 non-null    str     
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), str(5)
memory usage: 80.4 KB


In [57]:
# decoded = encoded.map(lambda x: encoder.inverse_transform([int(x)])[0] if pd.notnull(x) else np.nan)
# df_work[column + '_decoded'] = decoded

In [58]:
encoded_df = test_output['encoded']

In [60]:
decode_output

,sex_decoded,embarked_decoded,who_decoded,embark_town_decoded,alive_decoded
0,male,S,man,Southampton,no
1,female,C,woman,Cherbourg,yes
2,female,S,woman,Southampton,yes
3,female,S,woman,Southampton,yes
4,male,S,man,Southampton,no
...,...,...,...,...,...
886,male,S,man,Southampton,no
887,female,S,woman,Southampton,yes
888,female,S,woman,Southampton,no
889,male,C,man,Cherbourg,yes
